In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [2]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# Create a Holistic model instance
holistic = mp_holistic.Holistic(
    static_image_mode=False, 
    min_detection_confidence=0.5, 
    min_tracking_confidence=0.5
)

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
    image.flags.writeable = False                  
    results = model.process(image)                
    image.flags.writeable = True                   
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) 
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) 
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) 

In [21]:
def draw_styled_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,222,726), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
 
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [38]:
cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        ret, frame = cap.read()
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        draw_styled_landmarks(image, results)
        cv2.imshow('OpenCV Feed', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [39]:
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

In [40]:
lh

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [41]:
rh

array([ 2.20799536e-01,  9.19735789e-01,  5.10148027e-07,  3.06297868e-01,
        8.93291116e-01, -3.20486911e-02,  3.78917515e-01,  8.23510408e-01,
       -5.08930422e-02,  4.32007879e-01,  7.64581978e-01, -6.89982101e-02,
        4.83525217e-01,  7.17667401e-01, -8.83226097e-02,  3.55483711e-01,
        6.78083539e-01, -3.86699028e-02,  3.98593247e-01,  5.86800158e-01,
       -7.73009807e-02,  4.32142943e-01,  5.48519194e-01, -1.06576070e-01,
        4.68975425e-01,  5.27719378e-01, -1.26189217e-01,  3.07288051e-01,
        6.57522738e-01, -4.67651412e-02,  3.31455380e-01,  5.53324699e-01,
       -8.43861029e-02,  3.68865818e-01,  5.16974032e-01, -1.12200990e-01,
        4.09751475e-01,  5.02131045e-01, -1.31135613e-01,  2.54675806e-01,
        6.67270601e-01, -5.98685369e-02,  2.73470134e-01,  5.64037800e-01,
       -9.75521803e-02,  3.06949556e-01,  5.19419730e-01, -1.21674664e-01,
        3.46527368e-01,  4.93461281e-01, -1.37151226e-01,  2.00600475e-01,
        6.99528992e-01, -

In [42]:
def extract_keypoints(results):
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh, rh])

In [43]:
result_test = extract_keypoints(results)

In [44]:
result_test

array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  

In [48]:

DATA_PATH = os.path.join('MP_Data') 
actions = np.array(['friend','love','more','pain','play','stand','stop','what','front','right','left','up','down','now','eat','drink','super','hug','me','name','hello'])
no_sequences = 15
sequence_length = 15

In [46]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Process frame
    image, results = mediapipe_detection(frame, holistic)

    # Draw landmarks
    draw_landmarks(image, results)

    # Display the processed image
    cv2.imshow('Holistic Model Feed', image)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


ValueError: _graph is None in SolutionBase

In [ ]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for action in actions:
        for sequence in range(no_sequence):
            for frame_num in range(sequence_length):
                ret, frame = cap.read()
                if not ret:
                    print("Failed to capture frame.")
                    continue

                image, results = mediapipe_detection(frame, holistic)

                draw_landmarks(image, results)

                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, f'Collecting frames for {action} Video Number {sequence}', (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else:
                    cv2.putText(image, f'Collecting frames for {action} Video Number {sequence}', (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                
                cv2.imshow('OpenCV Feed', image)
                
                if cv2.waitKey(100) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [49]:

for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [50]:
cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    for action in actions:

        for sequence in range(no_sequences):

            for frame_num in range(sequence_length):


                ret, frame = cap.read()


                image, results = mediapipe_detection(frame, holistic)
#                 print(results)


                draw_styled_landmarks(image, results)
                

                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)

                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                

                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)


                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
cap.release()
cv2.destroyAllWindows()

In [51]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [52]:

label_map = {label:num for num, label in enumerate(actions)}

In [53]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [64]:
np.array(sequences).shape

(315, 15, 126)

In [65]:
np.array(labels).shape

(315,)

In [66]:
X = np.array(sequences)

In [67]:
X.shape

(315, 15, 126)

In [68]:
y = to_categorical(labels).astype(int)

In [69]:
y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

In [70]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [71]:
y_test.shape

(16, 21)

In [116]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input
from tensorflow.keras.callbacks import TensorBoard

In [117]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [118]:
model = Sequential()
model.add(Input(shape=(15, 126)))  
model.add(LSTM(64, return_sequences=True, activation='relu'))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(y_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [119]:
model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_21 (LSTM)                  │ (None, 15, 64)         │        48,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_22 (LSTM)                  │ (None, 15, 128)        │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_23 (LSTM)                  │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 21)             │           693 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 204,053 (797.08 KB)

 Trainable params: 204,053 (797.08 KB)

 Non-trainable params: 0 (0.00 B)

In [120]:

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)




X_train shape: (299, 15, 126)
y_train shape: (299, 21)


In [121]:
res = [.7, 0.2, 0.1]

In [122]:
actions[np.argmax(res)]

'friend'

In [123]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [124]:
X_train

array([[[ 6.79295361e-01,  8.15806389e-01,  3.88640700e-07, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 5.54366946e-01,  7.20835686e-01,  2.13475229e-08, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 5.50993323e-01,  7.24442363e-01,  1.53091349e-07, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        ...,
        [ 5.50030053e-01,  7.30404258e-01,  1.06615325e-07, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 5.52778125e-01,  7.31095016e-01,  1.00650716e-07, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 5.51547825e-01,  7.31610239e-01,  1.19486643e-07, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

       [[ 6.78590000e-01,  7.91787088e-01, -4.12794975e-07, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 6.87602341e-01,  9.40737844e-01, -2.22801901e-07, ...,
          0.00000000e+00,  0.00000000e

In [125]:
print("Shape of X_train:", X_train.shape)  # This should print (num_samples, 30, 1662)
print("Shape of y_train:", y_train.shape)  # This should print (num_samples,)

Shape of X_train: (299, 15, 126)
Shape of y_train: (299, 21)


In [126]:
model.fit(X_train, y_train, epochs=200, validation_split=0.2, batch_size=32)

Epoch 1/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 10s 243ms/step - categorical_accuracy: 0.0630 - loss: 3.0412 - val_categorical_accuracy: 0.0167 - val_loss: 3.0341
Epoch 2/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - categorical_accuracy: 0.0574 - loss: 3.0028 - val_categorical_accuracy: 0.0167 - val_loss: 2.9914
Epoch 3/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - categorical_accuracy: 0.0967 - loss: 2.9235 - val_categorical_accuracy: 0.0167 - val_loss: 3.4684
Epoch 4/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - categorical_accuracy: 0.1136 - loss: 2.8074 - val_categorical_accuracy: 0.0500 - val_loss: 2.9036
Epoch 5/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - categorical_accuracy: 0.1291 - loss: 2.7708 - val_categorical_accuracy: 0.0500 - val_loss: 2.9385
Epoch 6/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - categorical_accuracy: 0.1225 - loss: 2.6774 - val_categorical_accuracy: 0.0667 - val_loss: 2.8415
Epoch 7/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - categorical_accuracy: 0.1241 - loss: 2.6162 - 

In [127]:
res = model.predict(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 877ms/step


In [128]:
actions[np.argmax(res[4])]

'left'

In [129]:
actions[np.argmax(y_test[4])]

'left'

In [130]:
model.save('dasa.keras')

In [131]:

del model

In [133]:
model = load_model('dasa.keras')


In [134]:

loss, accuracy = model.evaluate(X_test, y_test)
print("Model accuracy:", accuracy)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - categorical_accuracy: 0.7500 - loss: 4.7076
Model accuracy: 0.75


In [135]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [136]:
yhat = model.predict(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 810ms/step


In [137]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [138]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[14,  0],
        [ 1,  1]],

       [[15,  1],
        [ 0,  0]],

       [[15,  0],
        [ 0,  1]],

       [[14,  0],
        [ 1,  1]],

       [[15,  0],
        [ 0,  1]],

       [[13,  1],
        [ 1,  1]],

       [[15,  0],
        [ 0,  1]],

       [[14,  0],
        [ 0,  2]],

       [[14,  2],
        [ 0,  0]],

       [[15,  0],
        [ 0,  1]],

       [[14,  0],
        [ 0,  2]],

       [[14,  0],
        [ 1,  1]]], dtype=int64)

In [ ]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
from itertools import cycle

def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    color_cycle = cycle(colors)  # Cycle through colors if not enough colors provided
    
    for num, prob in enumerate(res):
        # Ensure prob is a scalar
        prob = np.max(prob) if isinstance(prob, np.ndarray) else prob

        print("Type of prob:", type(prob), "Value:", prob)

        # Scale probability to image width for visualization
        bar_width = int(prob * 100)
        bar_height = 30
        top_left = (0, 60 + num * bar_height)
        bottom_right = (bar_width, 80 + num * bar_height)

        # Get the next color from the cycle
        current_color = next(color_cycle)

        # Draw rectangle and text
        cv2.rectangle(output_frame, top_left, bottom_right, current_color, -1)
        cv2.putText(output_frame, f"{actions[num]}: {prob:.2f}", (0, 75 + num * bar_height), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    
    return output_frame

In [2]:
plt.figure(figsize=(18, 18))
plt.imshow(cv2.cvtColor(prob_viz(res, actions, image, colors), cv2.COLOR_BGR2RGB))
plt.show()

NameError: name 'plt' is not defined

In [1]:
sequence = []
sentence = []
threshold = 0.8

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

NameError: name 'cv2' is not defined

In [161]:
res[np.argmax(res)] > threshold

True

In [162]:
(num_sequences,30,1662)

NameError: name 'num_sequences' is not defined

In [1]:
model.predict(np.expand_dims(X_test[0], axis=0))

NameError: name 'model' is not defined

In [ ]:
array([[5.532469e-04, 9.974770e-01, 1.969744e-03]], dtype=float32)